In [4]:
from funcat import *
from tushare import *
from rqalpha import *
from pytdx import *
from talib import *

In [2]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

np.seterr(all='ignore')
rcParams['figure.figsize'] = (14, 6)

In [3]:
from funcat.data.tushare_backend import TushareDataBackend
from funcat.data.rqalpha_data_backend import RQAlphaDataBackend

backend = "rqalpha"

if backend == "rqalpha":
    set_data_backend(RQAlphaDataBackend())
elif backend == "tushare":
    set_data_backend(TushareDataBackend())

set_start_date("2015-01-01")
S("000001.XSHG")  # 设置当前关注股票
T("2016-06-01")   # 设置当前观察日期

ModuleNotFoundError: No module named 'talib'

## 基本API

In [ ]:
print(O, H, L, C, V)  # 打印 Open High Low Close

In [ ]:
C / C[1] - 1  # 当天涨幅

In [ ]:
MA(C, 60)  # 打印60日均线

In [ ]:
C > MA(C, 60)  # 判断收盘价是否大于60日均线

In [ ]:
HHV(H, 30)  # 30日最高价

In [ ]:
COUNT(C > MA(C, 60), 30)  # 最近30日，收盘价 Close 大于60日均线的天数

In [ ]:
CROSS(MA(C, 10), MA(C, 20))  # 10日均线上穿

In [ ]:
COUNT(C > O, 10)  # 最近10天收阳线的天数

In [ ]:
HHV(MAX(O, C), 60)  # 最近60天K线实体的最高价

In [ ]:
#CROSS(MA(C, 5), MA(C, 10))  # 5日均线上穿10日均线

## 自定义公式

### 0x01 KDJ指标

In [ ]:
N, M1, M2 = 9, 3, 3

RSV = (CLOSE - LLV(LOW, N)) / (HHV(HIGH, N) - LLV(LOW, N)) * 100
K = EMA(RSV, (M1 * 2 - 1))
D = EMA(K, (M2 * 2 - 1))
J = K * 3 - D * 2

ax = plt.subplot()

plt.plot(K.series, label="K", linewidth=2)
plt.plot(D.series, label="D", alpha=0.7)
plt.plot(J.series, label="J", alpha=0.7)

print(K, D, J)
plt.legend(loc="best")

### 0x02 均线金叉死叉

In [ ]:
ax = plt.subplot()

ma10 = MA(C, 10)
ma30 = MA(C, 30)
buy_signal = CROSS(ma10, ma30)
sell_signal = CROSS(ma30, ma10)

plt.plot(C.series, label="close", linewidth=2)
plt.plot(ma10.series, label="ma10", alpha=0.7)
plt.plot(ma30.series, label="ma30", alpha=0.7)

plt.plot(np.where(buy_signal.series)[0], C.series[np.where(buy_signal.series)[0]], "^", label="buy", markersize=12, color="red")
plt.plot(np.where(sell_signal.series)[0], C.series[np.where(sell_signal.series)[0]], "v", label="sell", markersize=12, color="green")
plt.legend(loc="best")

### 0x03 DMA指标（Different of Moving Average）
[DMA指标](http://wiki.mbalib.com/wiki/DMA)。DMA指标（Different of Moving Average）又叫平行线差指标，是目前股市分析技术指标中的一种中短期指标，它常用于大盘指数和个股的研判。

In [ ]:
M1 = 10
M2 = 50
M3 = 10

DDD = MA(CLOSE, M1) - MA(CLOSE, M2)
AMA = MA(DDD, M3)

print(DDD, AMA)

f, (ax1, ax2) = plt.subplots(2, 1)

ax1.plot(C.series, label="close")
ax1.plot(MA(C, 30).series, label="ma60")
ax1.set_xlim(50)

ax2.plot(DDD.series, label="DDD")
ax2.plot(AMA.series, label="AMA")
ax2.set_xlim(50)

plt.legend(loc="best")

## 选股

In [ ]:
# 选出涨停股
select(
    lambda : C / C[1] - 1 >= 0.0995,
    start_date=20170104,
	end_date=20170104,
)

In [ ]:
# 选出最近30天K线实体最高价最低价差7%以内，最近100天K线实体最高价最低价差25%以内，
# 最近10天，收盘价大于60日均线的天数大于3天
select(
    lambda : ((HHV(MAX(C, O), 30) / LLV(MIN(C, O), 30) - 1 < 0.07)
              & (HHV(MAX(C, O), 100) / LLV(MIN(C, O), 100) - 1 > 0.25)
              & (COUNT(C > MA(C, 60), 10) > 3)
             ),
    start_date=20170104,
	end_date=20170104,
)

In [ ]:
# 选出最近3天每天的成交量小于20日成交量均线，最近3天最低价低于20日均线，最高价高于20日均线
# 自定义选股回调函数
def callback(date, order_book_id, symbol):
    print("Cool, 在", date, "选出", order_book_id, symbol)

    
select(
    lambda : (EVERY(V < MA(V, 20) / 2, 3) & EVERY(L < MA(C, 20), 3) & EVERY(H > MA(C, 20), 3)),
    start_date=20170104,
	end_date=20170104,
    callback=callback,
)